In [14]:
import yaml
import numpy as np
from transformers import pipeline
from gorse import Gorse

In [2]:
# Load classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [21]:
# Load labels from YAML
with open('labels.yml') as f:
    content = yaml.load(f, yaml.Loader)
    labels = list(content.keys())

In [25]:
client = Gorse('http://127.0.0.1:8088', '')
count = 0
cursor = ''
while count < 10:
    items, cursor = client.get_items(10, cursor)
    for item in items:
        if len(item['Comment']) > 0 and (item['Labels'] is None or len(item['Labels']) <= 1):
            count += len(items)
            print(item['ItemId'], '\t', item['Comment'], '\t', item['Labels'])
            # Find top k labels
            result = classifier(item['Comment'], labels)
            sequence = result['sequence']
            labels = np.array(result['labels'])
            scores = np.array(result['scores'])
            top_k = np.argsort(-scores)[:5]
            print(scores[top_k])
            print(labels[top_k])
    if cursor == '':
        break

00-evan:shattered-pixel-dungeon 	 Traditional roguelike game with pixel-art graphics and simple interface 	 ['java']
[0.18064795 0.1311568  0.01668721 0.01612063 0.01601641]
['pixel-art' 'game' 'awesome' 'okr' 'esp']
007:hashcash-js 	 Generic javascript implementation of hashcash 	 ['javascript']


KeyboardInterrupt: 